In [ ]:
# pip install faiss-cpu
# pip install torch
# pip install tensorflow
# pip install transformers

In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import tensorflow as tf

/Users/ktg/miniforge3/envs/fastapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [3]:
text = 'You passed along `num_labels=3` with an incompatible id to label map'

In [4]:
raw_input_ids = tokenizer.encode(text)

In [5]:
raw_input_ids

[16879,
 17884,
 17065,
 19542,
 17223,
 299,
 1700,
 15585,
 24687,
 15254,
 309,
 22549,
 294,
 17422,
 297,
 16997,
 314,
 260,
 250,
 295,
 20676,
 304,
 16160,
 1700,
 15195,
 22692,
 27764,
 311,
 296,
 15555,
 297,
 16203,
 1700,
 18509,
 27141,
 1700,
 17422,
 297,
 16997,
 19281,
 20924]

In [3]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import torch

class PseudoTextSearchSystem:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
        self.model = AutoModel.from_pretrained("kykim/bert-kor-base")
        self.index = None
        self.text_db = []
        
    def add_text_to_db(self, text):
        self.text_db.append(text)
        
    def build_index(self):
        embeddings = self._get_embeddings(self.text_db)
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)
        
    def search_text(self, query, k=5):
        if self.index is None:
            raise ValueError("Index is not built yet. Call build_index() first.")
        
        query_embedding = self._get_embeddings([query])[0]
        distances, indices = self.index.search(np.array([query_embedding]), k)
        results = [(self.text_db[idx], score) for idx, score in zip(indices[0], distances[0])]
        return results
    
    def _get_embeddings(self, texts):
        encoded_input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
            embeddings = model_output.last_hidden_state[:, 0, :].numpy()
        return embeddings

In [4]:
# Create the search system
search_system = PseudoTextSearchSystem()

# Add texts to the database
search_system.add_text_to_db("안녕하세요, 반갑습니다.")
search_system.add_text_to_db("오늘 날씨가 좋네요.")
search_system.add_text_to_db("저는 파이썬 개발자입니다.")

# Build the search index
search_system.build_index()

# Search for similar texts
results = search_system.search_text("안녕하세요, 좋은 하루 되세요.")
print(results)

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('안녕하세요, 반갑습니다.', 296.9377), ('저는 파이썬 개발자입니다.', 545.05164), ('오늘 날씨가 좋네요.', 586.89777), ('저는 파이썬 개발자입니다.', 3.4028235e+38), ('저는 파이썬 개발자입니다.', 3.4028235e+38)]
